In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vapeplot 
from scipy import interp
import scipy.stats
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

#### Goal
--------------------

The goal of this project is to make a classifier that predicts the final rankings for bakers.
The idea is to make a model for each episode and to use data from previous episodes in the model.
Therefore, a classifier for episode 1 will likely be bad at predicting the final outcome, but a classifier for episode 5 might accurately predict who will be in the top 3 and who might be eliminated in the next episode



In [7]:
from datetime import datetime
def timestamp(): return datetime.today().strftime('%Y%m%d')

from sklearn.preprocessing import QuantileTransformer
def quantile_scale(df,feats):
    qua = df
    scaler = QuantileTransformer(
        n_quantiles=10,
        random_state=42,
        ignore_implicit_zeros=True, #sparse matrix
    )
    # fit the scaler
    scaler.fit(qua[feats])
    # transform values
    qua[feats] = scaler.transform(qua[feats])
    return qua
def tiered(classes):
    trans = []
    for x in classes:
        if x==1: c=0
        if x==2: c=1
        if x>=3 and x<=4: c=2
        if x>=5 and x<=7: c=3
        if x>=8: c=4
        trans.append(c)
    return trans

In [8]:
merge_col = ['season','baker','index','episode','place']
tech = pd.read_csv("../RESULTS/gbbo.techinical.data.s10.20191104.tsv",sep="\t")
star = pd.read_csv("../RESULTS/gbbo.starbaker.data.s10.final.tsv",sep="\t")
gbbo = pd.merge(tech, star,  how='left', left_on=merge_col, right_on =merge_col)
gbbo = gbbo[['season','baker','episode','place','tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']]
gbbo.to_csv("../RESULTS/gbbo.features.s10.final.tsv".format(timestamp()),sep="\t",index=False)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
0,10,Alice,1,0,5.00,5,0.00,0,0.00,0,0.00,0
1,10,Alice,2,0,3.00,1,0.50,1,0.50,1,0.00,0
2,10,Alice,3,0,4.33,7,0.33,0,0.33,0,0.33,1
3,10,Alice,4,0,5.25,8,0.25,0,0.25,0,0.25,0
4,10,Alice,5,0,5.40,6,0.20,0,0.20,0,0.20,0


In [9]:
gbbo = pd.read_csv("../RESULTS/gbbo.features.s10.final.tsv",sep="\t")
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
max_epi = max(gbbo['episode'])
gbbo = gbbo.loc[gbbo['episode']==max_epi]
gbbo = quantile_scale(gbbo,feats)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
9,10,Alice,10,0,0.897436,0.916667,0.904762,0.0,0.814815,0.0,0.277778,0.0
19,10,Amelia,10,10,0.111111,0.000000,0.000000,0.0,0.000000,0.0,0.666667,0.0
29,10,Dan,10,12,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.277778,0.0
39,10,David,10,0,0.518519,0.833333,0.000000,0.0,0.909091,0.0,0.277778,0.0
49,10,Helena,10,8,0.866667,0.000000,0.000000,0.0,0.000000,0.0,0.277778,0.0


In [10]:
tech = pd.read_csv("../RESULTS/gbbo.features.20190909.tsv",sep='\t')
tech = tech.loc[tech['episode']==max_epi]
qua = quantile_scale(tech,feats)
qua['place']=tiered(qua['place'])
X, y = np.matrix(qua[feats]), np.array(qua['place'])

In [17]:
pd.concat(
    [
        pd.read_csv("../RESULTS/gbbo.features.20190909.tsv",sep='\t'),
        pd.read_csv("../RESULTS/gbbo.features.s10.final.tsv",sep="\t")
    ]
).to_csv("../RESULTS/deepbake_features.20201016.tsv",sep="\t")

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.activations import relu

def create_model( nl1=1, nl2=1,  nl3=1, 
                 nn1=1000, nn2=500, nn3 = 200, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0,input_shape=None,output_shape=None):    
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture
    https://www.kaggle.com/arrogantlymodest/randomised-cv-search-over-keras-neural-network
    '''
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
    model = Sequential()
    first=True  
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))       
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model
##################################
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
tech = pd.read_csv("../RESULTS/gbbo.features.20190909.tsv",sep='\t')
tech['place']=tiered(tech['place'])
input_shape = len(feats)
output_shape = len(set(tech['place']))


l1 = 0.0001
l2 = 0.0001
lr = 0.0001
nl1 = 1
nl2 = 1
nl3 = 1
nn1 = 800
nn2 = 800
nn3 = 300

dropout = 0.1
decay = 1e-09
act='relu'
n_dims = len(feats)
n_classes = len(set(tech['place']))


BATCH,EPOCHS = 12, 25
####
GBBO = pd.read_csv("../RESULTS/gbbo.features.s10.final.tsv",sep="\t")

for e in set(GBBO['episode']):
    gbbo = GBBO.loc[GBBO['episode']==e]
    gbbo = quantile_scale(gbbo,feats)
    test = np.matrix(gbbo[feats])

    tech = pd.read_csv("../RESULTS/gbbo.features.20190909.tsv",sep='\t')
    tech = tech.loc[tech['episode']==e]
    qua = quantile_scale(tech,feats)
    qua['place']=tiered(qua['place'])
    
    X, y = np.matrix(qua[feats]), np.array(qua['place'])
    
    nn = create_model( nl1=nl1, nl2=nl2,  nl3=nl3, 
                     nn1=nn1, nn2=nn2, nn3 = nn3, 
                     lr=lr, decay=decay, l1=l1, l2=l2,
                     act = act, dropout=dropout,
                     input_shape=n_dims,
                     output_shape=n_classes)
    
    nn.fit(X,y,validation_split=0., batch_size=BATCH, epochs=EPOCHS,verbose=0)

    preds = nn.predict_classes(test)
    probs = nn.predict_proba(test)
    gbbo['preds']=preds
    # probability baker is a finalist
    top = probs[:,0]
    # probability baker is a finalist or a runner-up
    top3 = probs[:,0]+probs[:,1]
    # bottom tier (8th and below)
    bot = probs[:,-1]
    # 5th - 7th
    nextbot = probs[:,-2]
    third = probs[:,-3]

    gbbo['bottom']=np.round(bot*100,decimals=2)
    gbbo['finalist']=np.round(top*100,decimals=2) 
    gbbo['top3'] = np.round(top3*100,decimals=2)
    gbbo['fifthseventh'] = np.round(nextbot*100,decimals=2)
    gbbo['thirdforth'] = np.round(third*100,decimals=2)

    gbbo.to_csv("../RESULTS/gbbo.techinical.s10.week{}.final2.keras.preditions.txt".format(e),sep="\t",index=False)
    sub = ['baker','preds','finalist','top3','bottom','fifthseventh','thirdforth']
    print(e)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use `model.predict()` instead.
1
2
3
4
5
6
7
8
9
10


In [15]:
gbbo = gbbo[sub].sort_values(by=['finalist'],ascending=False)
print(gbbo)

        baker  preds   finalist       top3     bottom  fifthseventh  \
39      David      1  34.000000  87.599998   2.700000      5.310000   
129     Steph      1  23.459999  98.190002   0.240000      0.900000   
9       Alice      1  22.049999  97.610001   0.260000      1.210000   
89   Michelle      2  11.310000  19.730000   9.740000     34.250000   
29        Dan      4   3.840000   6.910000  61.919998     19.610001   
99       Phil      4   3.370000   6.140000  49.419998     29.990000   
49     Helena      3   2.850000   5.580000  23.320000     52.799999   
59      Henry      2   2.240000   3.340000  18.580000     38.310001   
79    Michael      3   1.550000   2.630000   4.970000     48.189999   
19     Amelia      4   1.000000   1.670000  73.669998     17.209999   
69      Jamie      4   1.000000   1.670000  73.669998     17.209999   
119     Rosie      3   0.550000   0.870000  35.009998     36.650002   
109     Priya      4   0.530000   0.900000  53.860001     34.599998   

     